# Harris County Home Price Estimations

In [1]:
import sqlite3

import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


In [2]:
con = sqlite3.connect('HouseProtestValues.db')

sql_query = '''SELECT  br.acct,
                       br.bld_num,
                       br.date_erected,
                       br.yr_remodel,
                       br.im_sq_ft,
                       br.gross_ar,
                       br.base_ar,
                       ra.land_ar,
                       br.perimeter,
                       br.size_index,
                       ra.land_val,
                       ra.bld_val,
                       ra.assessed_val,
                       ra.tot_appr_val,
                       ra.tot_mkt_val
            FROM building_res as br
            LEFT JOIN real_acct as ra ON br.acct = ra.acct
            WHERE br.impr_tp = 1001 AND br.property_use_cd = 'A1' AND br.date_erected > 10;'''

building_data = pd.read_sql_query(sql_query, con)

In [3]:
# Story Height Index: STY
# Room: Bedroom: RMB
# Room: Full Bath: RMF
# Room: Half Bath: RMH
# Room: Total: RMT
fixtures_sql = """SELECT *
                  FROM "fixtures"
                  WHERE type IN ('STY', 'RMB','RMF','RMH','RMT')
                """
fixtures = pd.read_sql_query(fixtures_sql, con)
fix_pt = fixtures.pivot_table(index=['acct', 'bld_num'], columns='type', values='units', aggfunc='sum')
fix_pt = fix_pt.reset_index()
fix_pt.fillna(0, inplace=True)

In [4]:
all_df = pd.merge(building_data, fix_pt, on=['acct', 'bld_num'], how='left')
all_df.dropna(inplace=True)

In [19]:
all_df.tail()

,acct,bld_num,date_erected,yr_remodel,im_sq_ft,gross_ar,base_ar,land_ar,perimeter,size_index,land_val,bld_val,assessed_val,tot_appr_val,tot_mkt_val,RMB,RMF,RMH,RMT,STY
1122301,1938003804013,1,2003,0,2117,2553,2117,5250,318,0.84,35438.0,233603.0,269041.0,269041.0,269041.0,3.0,2.0,1.0,7.0,2.0
1122302,1938003804014,1,2003,0,1611,2254,1611,5250,184,0.89,35438.0,204053.0,239491.0,166883.0,239491.0,3.0,2.0,0.0,6.0,1.0
1122303,1953050320690,1,1979,0,2168,2700,2168,6820,232,0.84,54560.0,201010.0,255570.0,240372.0,255570.0,3.0,2.0,0.0,5.0,1.0
1122304,1953080320060,1,1983,0,1723,2259,1723,7575,216,0.88,55740.0,173156.0,228896.0,228896.0,228896.0,3.0,2.0,0.0,6.0,1.0
1122305,1955020050280,1,1979,0,1374,2047,1374,8851,250,0.94,64580.0,160896.0,225476.0,216585.0,225476.0,3.0,2.0,1.0,5.0,2.0


In [20]:
x = all_df[['date_erected', 'yr_remodel', 'im_sq_ft',
     'gross_ar', 'base_ar', 'land_ar', 'perimeter', 'size_index',
     'RMB', 'RMF', 'RMH', 'RMT', 'STY']]
y = all_df['assessed_val']

In [21]:
print(f"All Data{all_df.shape} | x {x.shape} | y {y.shape}")

All Data(1121460, 20) | x (1121460, 13) | y (1121460,)


In [22]:
# # initialize the OneHotEncoder
# encoder = OneHotEncoder(drop='first')
# 
# #Fit and transform the categorical columns
# encoded = encoder.fit_transform(all_df[x])
# 
# df_results = pd.DataFrame.sparse.from_spmatrix(encoded)
# df_results.columns = encoder.get_feature_names_out(x_categorical)
# df_results.shape

In [23]:
# df_results.info()

In [24]:
# Free up memory
building_data = None
fix_pt = None
fixtures = None


In [25]:
## Reset index of both dataframes
# all_df.reset_index(drop=True, inplace=True)
# df_results.reset_index(drop=True, inplace=True)
# 
# # Merge the encoded data and continuous data together
# encoded_features = pd.concat([all_df[x_continuous], df_results], join='inner', axis=1)

In [26]:
# encoded_features.shape

In [27]:
# sns.histplot(data=all_df, x="assessed_val")

In [28]:
# sns.histplot(data=all_df, x="assessed_val", log_scale=True)

In [29]:
# sns.pairplot(data=all_df, vars=['assessed_val', 'date_erected', 'im_sq_ft', 'school_dist', 'Neighborhood_Code'])

In [30]:
# sns.pairplot(data=all_df, vars=['assessed_val', 'land_ar', 'RMB', 'RMF', 'RMH', 'STY'])

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [32]:
model = ExtraTreesRegressor(random_state=42)

In [ ]:
model_fit = model.fit(x_train, y_train)

In [ ]:
cross_val_score(model_fit, x_train, y_train, scoring='accuracy', cv=5, n_jobs=-1).mean()

In [21]:
y_pred = model.predict(x_test)
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")

C:\Users\Blake\miniconda3\envs\HouseProtestValues\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Mean Squared Error: 31645615157.76855


In [1]:
param_grid = {
    'n_estimators': [100, 250, 500],
    'min_samples_leaf': [5, 15, 25],
    'max_features': [10, 20, 30],
    'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
}

In [ ]:
etc2 = GridSearchCV(model_fit, param_grid, cv=3, n_jobs=-1)

In [ ]:
etc2.fit(x_train, y_train)

In [ ]:
etc2.best_params_

In [ ]:
etc2.best_score_